In [ ]:
!pip install ../input/efficientnet-pytorch-070/efficientnet_pytorch-0.7.0-py3-none-any.whl

In [ ]:
import os

import albumentations
import numpy as np
import pandas as pd
from PIL import Image
from scipy.special import softmax

import torch
import torch.nn as nn
from torchvision import models, transforms


from efficientnet_pytorch import EfficientNet

In [ ]:
resnet_model_path = "../input/rn-wc-tta-calr-clahe-cutmix/model(24).pth"
effnet_model_path = "../input/en-b4-tta-calr-clahe-v3/eff_epoch_11.pth"

sample_sub_path = "../input/cassava-leaf-disease-classification/sample_submission.csv"
test_images_path = "../input/cassava-leaf-disease-classification/test_images"

# Creating model and loading weights

In [ ]:
# # Creating the resnet model
resnet_model = models.resnext50_32x4d(pretrained=False)
resnet_model.fc = nn.Linear(2048, 5)

resnet_model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
resnet_model.load_state_dict(torch.load(resnet_model_path))
resnet_model.eval()

# # Creating the efficientnet model
effnet_model = EfficientNet.from_name('efficientnet-b4', num_classes=5)
effnet_model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
effnet_model.load_state_dict(torch.load(effnet_model_path))
effnet_model.eval()

# Augmentations

In [ ]:
sub_aug = albumentations.Compose([
                albumentations.RandomResizedCrop(512, 512, scale=(0.5, 1.0)),
                albumentations.Transpose(p=0.5),
                albumentations.HorizontalFlip(p=0.5),
                albumentations.VerticalFlip(p=0.5),
                albumentations.ShiftScaleRotate(p=0.8),
                albumentations.Normalize(
                    mean=[0.485, 0.456, 0.406], 
                    std=[0.229, 0.224, 0.225], 
                    max_pixel_value=255.0, 
                    p=1.0
                )
            ], p=1.)

# Creating the Submission file

In [ ]:
# resnet
sample_sub = pd.read_csv(sample_sub_path)
tta_count = 10

predictions = []
for _, sample_row in sample_sub.iterrows():
    
    image_pred = 0
    for j in range(tta_count):
        image = np.array(Image.open(os.path.join(test_images_path, sample_row.image_id)))
        image = sub_aug(image=image)["image"]
        image = transforms.ToTensor()(np.array(image))
        image = image.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        
        # resnet
        outputs = resnet_model(image.unsqueeze(0))
        image_pred += outputs.detach().cpu().numpy() * 0.6
        
        # effnet
        outputs = effnet_model(image.unsqueeze(0))
        image_pred += outputs.detach().cpu().numpy() * 0.4
    
    image_pred /= (tta_count)
    predictions.extend(image_pred)
    
predictions = softmax(predictions, axis=1).argmax(axis=1)

sub_df = pd.DataFrame({'image_id':sample_sub.image_id, 'label':predictions})
sub_df.to_csv('submission.csv', index=False)
print(sub_df)